## 1) MSE 방식으로 학습 -> 수정함
## 2) feature의 개수가 너무 많음 -> 7개 조절
## 3) layer 깊이 조절하기 -> 3개로 조절
## 4) Normalize 다른거 -> PCA 변수자체를 norm으로 만듬

In [17]:
#일반 라이브러리
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

#API를 사용하기 위한 import
# from geoband.API import *
# import folium
# import json
# import geopandas as gpd

# 전처리 및 operator
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

# model
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import RandomizedSearchCV

In [18]:
PCA_data = pd.read_csv("PCA.csv")
PCA_data = PCA_data.iloc[:,1:]

data = pd.read_csv("total.csv")
data = data.iloc[:,1:]


In [19]:
data.columns.tolist()[-5:]

[' rob_satety_q1',
 'mur_safety_q2',
 'ta_safety_q3',
 'raw_odder_q4',
 'overall_q5']

In [20]:
target = "overall_q5"

dataset = pd.concat([PCA_data[["PC1","PC2","PC3","PC4","PC5","PC6","PC7"]], data[["jur_stn",target]]], axis = 1)

train = dataset[dataset[target].isnull() == False]
test = dataset[dataset[target].isnull()]

print(train.shape)
print(test.shape)

(246, 9)
(82, 9)


In [21]:
train

PC1       PC2       PC3       PC4       PC5       PC6       PC7  \
0   -5.843327 -1.552553 -0.386392  0.054962  0.000691  2.365791  0.396099   
1   -5.753760 -1.934429 -0.455216  0.511299  0.112767  2.344618  0.208428   
2   -6.051162 -1.223132 -0.257158 -0.117475  0.057478  2.496029  0.342167   
3   -6.093965 -1.350174 -0.233333  0.005199  0.116385  2.718029  0.422335   
4   -5.929747 -0.562165  0.425840 -0.649937  0.049884  2.583079  0.319921   
..        ...       ...       ...       ...       ...       ...       ...   
321 -3.092893 -4.157176 -1.933148  1.088230  4.034375  0.099208 -2.121681   
322 -3.211471 -3.146838 -1.648940  0.706636  4.047320  0.200138 -1.773088   
323 -3.235498 -3.143853 -1.607412  0.673844  4.086958 -0.036664 -2.125947   
324 -3.110050 -2.502579 -1.075531  0.192931  4.602668 -0.905996 -2.867498   
325 -3.335185 -2.680719 -1.432199  0.578353  4.040644  0.207236 -1.910025   

     jur_stn  overall_q5  
0    마산동부경찰서        70.3  
1    마산동부경찰서        71.2  
2    마산동부경찰서        71.8  
3    마산동부경찰서        72.6  
4    마산동부경찰서        73.1  
..       ...         ...  
321  창원중부경찰서        70.8  
322  창원중부경찰서        74.2  
323  창원중부경찰서        72.9  
324  창원중부경찰서        74.1  
325  창원중부경찰서        76.1  

[246 rows x 9 columns]

In [22]:
X = train[["PC1","PC2","PC3","PC4","PC5","PC6","PC7"]]
Y = train[target]

# train & test 2
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state = 0, shuffle = True,
                                                    train_size = 0.83)

print(X_train.shape, Y_train.shape)
print(X_test.shape, Y_test.shape)

(204, 7) (204,)
(42, 7) (42,)


In [23]:
# 모델전처리1
# from sklearn.preprocessing import MinMaxScaler
# scaler = MinMaxScaler()
# scaler.fit(X_train)
# X_train = scaler.transform(X_train)
# X_test = scaler.transform(X_test)


# 모델전처리2

# from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler()
# scaler.fit(train)
# train = scaler.transform(train)

In [24]:
# step1) import
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader # custom_dataset & Dataloader

# step2) device
device = "cuda" if torch.cuda.is_available() else "cpu"
torch.manual_seed(777)
if device == "cuda":
    torch.cuda.manual_seed_all(777)
print(device)

cpu


In [25]:
X_test.shape[0]

42

In [26]:
X_train.shape[0] % 17
X_test.shape[0] % 14

0

In [27]:
# step3) hyper parameter
training_epoch = 10000  # epoch 증대
batch_size = 17       # 4개에서 증가시킴 --> 학습이 안정적
learning_rate = 0.01 # 0.005에서 줄임

In [28]:
# step4) dataset and dataloader
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self,X,Y):
        self.x_data = X
        self.y_data = Y
        
    def __len__(self):
        return len(self.x_data)
    
    def __getitem__(self, idx):
        x = self.x_data[idx]
        y = self.y_data[idx]        
        x = torch.FloatTensor([x])
        y = torch.FloatTensor([y])
        
        return x,y
        
# train
X_train = np.array(X_train)
Y_train = np.array(Y_train)
train_dataset = CustomDataset(X_train, Y_train)
train_loader = DataLoader(train_dataset, batch_size = batch_size, shuffle = True, drop_last = True)

# test
X_test = np.array(X_test)
Y_test = np.array(Y_test)
test_dataset = CustomDataset(X_test, Y_test)
test_loader = DataLoader(test_dataset, batch_size = 14, shuffle = False, drop_last = False)
X_test = torch.FloatTensor([X_test])
Y_test = torch.FloatTensor([Y_test])


In [29]:
# step5) model

class LinearModel(nn.Module):
    def __init__(self):
        super(LinearModel, self).__init__()
        
        # feature의 개수가 34개 (전체모델)
        # feature의 개수가 17개 (PCA모델)
        self.layer1 = nn.Sequential(
            nn.Linear(7,8),
            nn.ReLU(),
        )
            
        self.layer2 = nn.Sequential(
            nn.Linear(8,1),
        )
                
    def forward(self,x):
        out = self.layer1(x)
        out = self.layer2(out)        
        #out = self.layer3(out)
        return out
    
    def initalize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):  
                nn.init.kaiming_normal_(m.weight, mode = "fan_out", nonlinearity= "leaky_relu" ) # 321 -> 20 
                nn.init.constant_(m.bias, 0)
        print("Weight Init Success!")
            
model = LinearModel().to(device)
model.initalize_weights()


Weight Init Success!


In [30]:
# step6) loss & optim
criterion = nn.MSELoss().to(device)
optimizer = optim.Adam(model.parameters(), lr = learning_rate)
lr_sche = optim.lr_scheduler.StepLR(optimizer, step_size= 200, gamma=0.9) # 10에서 20으로 바꿈


In [31]:
loss_list = []
r2_list = []

iteration = len(train_loader)
for epoch in range(training_epoch):
    model.train()
    r2 = 0
    loss = 0
    
    lr_sche.step()
    for sample in train_loader:
        optimizer.zero_grad()

        # Data
        X, Y = sample
        X = X.to(device)
        Y = Y.to(device)
        Y.squeeze_()

        # hypothesis
        hypothesis = model(X)
        hypothesis.squeeze_()

        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        #calculate
        loss += cost.item()
        r2 += r2_score(Y,hypothesis.detach().numpy())

    loss /= iteration
    r2 /= iteration

    loss_list.append(loss)
    r2_list.append(r2)

    if (epoch+1) % 1000 == 0:
        print("[Epoch {:04d}] loss = {:.4f} r2 = {:.4f}".format(epoch+1, loss, r2))

    if (epoch+1) % 1000 == 0:
        with torch.no_grad():
            model.eval()
            test_loss = 0
            test_r2 = 0

            for X,Y in test_loader:
                X = X.to(device)
                Y = Y.to(device)
                Y.squeeze_()

                # forward
                hypothesis = model(X)
                hypothesis.squeeze_()

                test_loss += criterion(hypothesis, Y).item()
                test_r2 += r2_score(Y, hypothesis.detach().numpy())

            test_loss /= len(test_loader)
            test_r2 /= len(test_loader)
            print("[Test] loss = {:.4f} r2 = {:.4f}".format(test_loss, test_r2))
            print("\n")

print("Done")
            

/Users/igyuseog/opt/anaconda3/lib/python3.8/site-packages/torch/optim/lr_scheduler.py:118: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


[Epoch 1000] loss = 3.4414 r2 = 0.6000
[Test] loss = 4.0439 r2 = 0.3219


[Epoch 2000] loss = 3.2629 r2 = 0.6221
[Test] loss = 3.9111 r2 = 0.3387


[Epoch 3000] loss = 3.2743 r2 = 0.6132
[Test] loss = 3.9035 r2 = 0.3418


[Epoch 4000] loss = 3.2135 r2 = 0.6402
[Test] loss = 3.8843 r2 = 0.3448


[Epoch 5000] loss = 3.1865 r2 = 0.6216
[Test] loss = 3.8901 r2 = 0.3450


[Epoch 6000] loss = 3.1700 r2 = 0.6414
[Test] loss = 3.9397 r2 = 0.3359


[Epoch 7000] loss = 3.1725 r2 = 0.6302
[Test] loss = 3.9457 r2 = 0.3346


[Epoch 8000] loss = 3.1602 r2 = 0.6397
[Test] loss = 3.9407 r2 = 0.3357


[Epoch 9000] loss = 3.1595 r2 = 0.6244
[Test] loss = 3.9414 r2 = 0.3357


[Epoch 10000] loss = 3.1583 r2 = 0.6063
[Test] loss = 3.9467 r2 = 0.3347


Done


In [149]:
print(Y)
print(hypothesis)

tensor([71.2000, 77.7000, 73.5000, 67.9000, 71.0000, 69.9000, 72.3000, 73.3000,
        65.6000, 72.4000, 68.1000, 63.4000, 68.1000, 63.8000])
tensor([70.8447, 71.9116, 75.3496, 67.0268, 69.5330, 66.2083, 67.9208, 66.7531,
        73.8830, 70.1403, 67.1118, 65.6201, 69.8401, 68.6699])


In [150]:
torch.mean(abs(Y - hypothesis))

tensor(3.2366)

# q5


[Epoch 10000] loss = 3.0520 r2 = 0.6574 <br>
[Test] loss = 3.8111 r2 = 0.3510

In [68]:
t = torch.FloatTensor(np.array(test.iloc[:,:-2]))
q5 = model(t).squeeze().tolist()

In [73]:
q5 = pd.DataFrame(q5)
q5.head()

0
0  72.227943
1  72.491867
2  75.432129
3  75.802956
4  76.064537

# q1
[Epoch 5000] loss = 3.7710 r2 = 0.7012 <br>
[Test] loss = 5.2094 r2 = 0.2837

In [110]:
q1 = model(t).squeeze().tolist()
q1 = pd.DataFrame(q1)
q1.head()

0
0  74.652580
1  74.779327
2  77.768486
3  77.897026
4  80.347832

# q2

[Epoch 10000] loss = 3.7328 r2 = 0.6673 <br>
[Test] loss = 6.9457 r2 = 0.2891

In [111]:
q2 = model(t).squeeze().tolist()
q2 = pd.DataFrame(q2)
q2.head()

0
0  76.546989
1  76.633797
2  78.503731
3  78.437355
4  81.259010

# q3

[Epoch 10000] loss = 3.7243 r2 = 0.3939
[Test] loss = 8.2219 r2 = -0.5128

--> 성능이 굉장히 좋지 않아서 다중회귀분석 모델 사용

In [133]:
q3 = model(t).squeeze().tolist()
q3 = pd.DataFrame(q3)
q3.head()

0
0  68.790192
1  68.882507
2  71.309280
3  71.521454
4  68.954582

# q4

[Epoch 10000] loss = 7.1436 r2 = 0.5659
[Test] loss = 13.2010 r2 = 0.0416

같은 경우도 성능이 굉장히 좋지 않아서 앙상블 모델에서 데이터 가져오기

In [151]:
q4 = model(t).squeeze().tolist()
q4 = pd.DataFrame(q4)
q4.head()

0
0  67.609703
1  67.995857
2  75.961418
3  75.530128
4  71.116188

# 제출물 만들기

In [159]:
correct = pd.read_csv("./correct_answer.csv")
correct = correct.iloc[:,1:]
correct.columns = ["q1","q2","q3","q4","q5"]

Basic = pd.read_csv("./Basic.csv")
answers = pd.concat([Basic, correct], axis = 1)
answers.head()


jur_stn  year  상반기         q1         q2         q3         q4         q5
0  마산동부경찰서  2020    0  75.549856  77.942222  67.951587  68.941667  72.715301
1  마산동부경찰서  2020    1  75.761459  77.453333  67.203968  69.733333  72.519672
2  마산중부경찰서  2020    0  78.185317  79.773333  69.655556  71.608333  74.402186
3  마산중부경찰서  2020    1  78.187602  79.757778  69.623810  71.075000  74.236066
4  서울강남경찰서  2020    0  80.210797  81.506667  69.115079  72.266667  75.731148

In [164]:
q3_linear = pd.read_csv("q3.csv")
q3_linear = q3_linear.iloc[:,1:]

In [169]:
answers["q1"] = q1
answers["q2"] = q2
answers["q3"] = q3_linear
answers["q5"] = q5

In [172]:
answers = answers.drop(["year","상반기"], axis = 1)
answers["범죄 안전도"] = (answers["q1"] + answers["q2"]) /2
answers["분야별 안전도"] = answers["범죄 안전도"]*0.343 + answers["q3"]*0.305 + answers["q4"]*0.352
answers["종합체감안전도_점수"] = answers["q5"]*0.3 + answers["분야별 안전도"]*0.7

answers = answers[["jur_stn","종합체감안전도_점수", "분야별 안전도", "범죄 안전도", "q1","q2","q3","q4","q5"]]
answers.head()


jur_stn  종합체감안전도_점수    분야별 안전도     범죄 안전도         q1         q2         q3  \
0  마산동부경찰서   70.478066  69.728118  75.599785  74.652580  76.546989  64.032541   
1  마산동부경찰서   70.610993  69.804904  75.706562  74.779327  76.633797  63.250558   
2  마산중부경찰서   73.248077  72.312054  78.136108  77.768486  78.503731  66.574543   
3  마산중부경찰서   73.071024  71.900197  78.167191  77.897026  78.437355  65.804755   
4  서울강남경찰서   73.348605  72.184634  80.803421  80.347832  81.259010  62.397356   

          q4         q5  
0  68.941667  72.227943  
1  69.733333  72.491867  
2  71.608333  75.432129  
3  71.075000  75.802956  
4  72.266667  76.064537

In [174]:
submit = pd.read_csv("./25.결과제출양식.csv")
police = submit["경찰서명"]

In [175]:
for i in range(len(police)):
    temp = answers[answers["jur_stn"] == police[i].strip()].iloc[:,1:]
    submit.iloc[i,2:] = temp.iloc[0,:].tolist()

In [187]:
submit

순번        경찰서명  종합체감안전도_점수   \n(전반적안전도*0.3) + (분야별안전도*0.7)  \
0    1    서울중부경찰서                                    70.532638   
1    2    서울종로경찰서                                    73.883076   
2    3   서울남대문경찰서                                    72.735462   
3    4   서울서대문경찰서                                    74.825725   
4    5    서울혜화경찰서                                    70.588895   
5    6    서울용산경찰서                                    74.855564   
6    7    서울성북경찰서                                    78.001910   
7    8   서울동대문경찰서                                    72.048968   
8    9    서울마포경찰서                                    74.750065   
9   10   서울영등포경찰서                                    71.719919   
10  11    서울성동경찰서                                    71.123976   
11  12    서울동작경찰서                                    70.391146   
12  13    서울광진경찰서                                    71.577394   
13  14    서울서부경찰서                                    72.732469   
14  15    서울강북경찰서                                    72.413295   
15  16    서울금천경찰서                                    68.188812   
16  17    서울중랑경찰서                                    69.035065   
17  18    서울강남경찰서                                    73.348605   
18  19    서울관악경찰서                                    69.219090   
19  20    서울강서경찰서                                    70.432809   
20  21    서울강동경찰서                                    72.537762   
21  22    서울종암경찰서                                    72.022976   
22  23    서울구로경찰서                                    69.609199   
23  24    서울서초경찰서                                    75.448653   
24  25    서울양천경찰서                                    74.049888   
25  26    서울송파경찰서                                    73.540376   
26  27    서울노원경찰서                                    73.539378   
27  28    서울방배경찰서                                    74.560353   
28  29    서울은평경찰서                                    71.912273   
29  30    서울도봉경찰서                                    73.625636   
30  31    서울수서경찰서                                    74.678181   
31  32      세종경찰서                                    74.183602   
32  33    수원중부경찰서                                    69.560684   
33  34    수원남부경찰서                                    72.366290   
34  35    수원서부경찰서                                    71.313572   
35  36    창원중부경찰서                                    72.696517   
36  37    창원서부경찰서                                    72.994872   
37  38    마산중부경찰서                                    73.248077   
38  39    마산동부경찰서                                    70.478066   
39  40      진해경찰서                                    71.040120   
40  41      진주경찰서                                    73.993999   

    분야별 안전도\n(범죄안전도*0.343) + \n(교통사고안전도*0.305)\n + (법질서준수도*0.352)  \
0                                           70.720719               
1                                           73.585449               
2                                           72.752110               
3                                           73.587450               
4                                           70.749605               
5                                           74.721614               
6                                           76.354148               
7                                           71.698822               
8                                           73.754594               
9                                           71.620424               
10                                          70.650772               
11                                          69.926952               
12                                          71.163010               
13                                          73.058260               
14                                          72.439094               
15                                          68.349732               
16                                          68.831247     

In [197]:
submit.to_csv("./submit4.csv", encoding= "utf-8", index = False)


In [198]:
pd.read_csv("./submit4.csv")

순번        경찰서명  종합체감안전도_점수   \n(전반적안전도*0.3) + (분야별안전도*0.7)  \
0    1    서울중부경찰서                                    70.532638   
1    2    서울종로경찰서                                    73.883076   
2    3   서울남대문경찰서                                    72.735462   
3    4   서울서대문경찰서                                    74.825725   
4    5    서울혜화경찰서                                    70.588895   
5    6    서울용산경찰서                                    74.855564   
6    7    서울성북경찰서                                    78.001910   
7    8   서울동대문경찰서                                    72.048968   
8    9    서울마포경찰서                                    74.750065   
9   10   서울영등포경찰서                                    71.719919   
10  11    서울성동경찰서                                    71.123976   
11  12    서울동작경찰서                                    70.391146   
12  13    서울광진경찰서                                    71.577394   
13  14    서울서부경찰서                                    72.732469   
14  15    서울강북경찰서                                    72.413295   
15  16    서울금천경찰서                                    68.188812   
16  17    서울중랑경찰서                                    69.035065   
17  18    서울강남경찰서                                    73.348605   
18  19    서울관악경찰서                                    69.219090   
19  20    서울강서경찰서                                    70.432809   
20  21    서울강동경찰서                                    72.537762   
21  22    서울종암경찰서                                    72.022976   
22  23    서울구로경찰서                                    69.609199   
23  24    서울서초경찰서                                    75.448653   
24  25    서울양천경찰서                                    74.049888   
25  26    서울송파경찰서                                    73.540376   
26  27    서울노원경찰서                                    73.539378   
27  28    서울방배경찰서                                    74.560353   
28  29    서울은평경찰서                                    71.912273   
29  30    서울도봉경찰서                                    73.625636   
30  31    서울수서경찰서                                    74.678181   
31  32      세종경찰서                                    74.183602   
32  33    수원중부경찰서                                    69.560684   
33  34    수원남부경찰서                                    72.366290   
34  35    수원서부경찰서                                    71.313572   
35  36    창원중부경찰서                                    72.696517   
36  37    창원서부경찰서                                    72.994872   
37  38    마산중부경찰서                                    73.248077   
38  39    마산동부경찰서                                    70.478066   
39  40      진해경찰서                                    71.040120   
40  41      진주경찰서                                    73.993999   

    분야별 안전도\n(범죄안전도*0.343) + \n(교통사고안전도*0.305)\n + (법질서준수도*0.352)  \
0                                           70.720719               
1                                           73.585449               
2                                           72.752110               
3                                           73.587450               
4                                           70.749605               
5                                           74.721614               
6                                           76.354148               
7                                           71.698822               
8                                           73.754594               
9                                           71.620424               
10                                          70.650772               
11                                          69.926952               
12                                          71.163010               
13                                          73.058260               
14                                          72.439094               
15                                          68.349732               
16                                          68.831247     